
<h1 style="text-align:center; margin-top:5em">Neural Networks - Dr. Kfir Bar<h1>
<h2 style="text-align:center;font-style: italic;">"Reconstruct images from magnetic resonance imaging (MRI)<h2>

<p style="margin:3em 0 0.2em 5em">Members:<p>
<h3 style="margin-left:12em">Nimrod Feldman (311129555)<h3>
<h3 style="margin-left:12em">Ron Maayan ()<h3>
<h3 style="margin-left:12em">Segev Efraim ()<h3>

### U-net model
U-net is an autoencoder with skip connections in order to localize features more precisely for the upsampling path.
There is a similar study about reconstructing MRI images from undersampled k-space data in the web. A web page mention different operations applied throughtout the model they propose. 


# Implementation 



#### Model

Changes has been made to the data loader get_epoch_batch in order to have consisted widths across the train dataset. 
Then, we cropped the images to 320x320 size, so that it will easier to feed to the neural network we built.

In [1]:
import h5py, os
from functions import transforms as T
from functions.subsample import MaskFunc
from scipy.io import loadmat
from torch.utils.data import DataLoader
import numpy as np
import torch
from matplotlib import pyplot as plt
from torch.nn import functional as F
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, random_split, Subset
from torchvision import transforms
from torchvision.utils import save_image
device = 'cuda' if torch.cuda.is_available() else 'cpu'  # check whether a GPU is available
from skimage.measure import compare_ssim
import sys
def ssim(gt, pred):
    """ Compute Structural Similarity Index Metric (SSIM). """
    return compare_ssim(
        gt.transpose(1, 2, 0), pred.transpose(1, 2, 0), multichannel=True, data_range=gt.max()
    )
batch_size = 8
current_mask = 8
PATH = "unet" # path for the model saved paramters

In [2]:
class MRIDataset(DataLoader):
    def __init__(self, data_list, acceleration, center_fraction, use_seed):
        self.data_list = data_list
        self.acceleration = acceleration
        self.center_fraction = center_fraction
        self.use_seed = use_seed

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, index):
        subject_id = self.data_list[index]
        return get_epoch_batch(subject_id, self.acceleration, self.center_fraction, self.use_seed)

def get_epoch_batch(subject_id, acc, center_fract, use_seed=True):
    ''' random select a few slices (batch_size) from each volume'''

    fname, rawdata_name, slice = subject_id
    
    with h5py.File(rawdata_name, 'r') as data:
        rawdata = data['kspace'][slice]
                      
    slice_kspace = T.to_tensor(rawdata).unsqueeze(0)
    S, Ny, Nx, ps = slice_kspace.shape
    m = nn.ZeroPad2d(((512-Nx)//2, (512-Nx) // 2, 0, 0))
    # we're adding padding on the width to have it consisted in our data set
    slice_kspace = slice_kspace.permute(0,3,1,2)
    slice_kspace = m(slice_kspace)
    slice_kspace = slice_kspace.permute(0,2,3,1)
    S, Ny, Nx, ps = slice_kspace.shape
    

    # apply random mask
    shape = np.array(slice_kspace.shape)
    mask_func = MaskFunc(center_fractions=[center_fract], accelerations=[acc])
    seed = None if not use_seed else tuple(map(ord, fname))
    mask = mask_func(shape, seed)
      
    # undersample
    masked_kspace = torch.where(mask == 0, torch.Tensor([0]), slice_kspace)
    masks = mask.repeat(S, Ny, 1, ps)

    img_gt, img_und = T.ifft2(slice_kspace), T.ifft2(masked_kspace)

    # perform data normalization which is important for network to learn useful features
    # during inference there is no ground truth image so use the zero-filled recon to normalize
    norm = T.complex_abs(img_und).max()
    if norm < 1e-6: norm = 1e-6
    
    # normalized data
    img_gt, img_und, rawdata_und = img_gt/norm, img_und/norm, masked_kspace/norm
    img_gt = T.complex_center_crop(img_gt.squeeze(0), [320,320])
    img_und = T.complex_center_crop(img_und.squeeze(0), [320,320])
        
    return img_gt, img_und, rawdata_und.squeeze(0), masks.squeeze(0), norm

def load_data_path(train_data_path, val_data_path):
    """ Go through each subset (training, validation) and list all 
    the file names, the file paths and the slices of subjects in the training and validation sets 
    """

    data_list = {}
    train_and_val = ['train', 'val']
    data_path = [train_data_path, val_data_path]
      
    for i in range(len(data_path)): # 0: train_path , 1: val_path
        print("dataset-loader: opening ... ", data_path[i])

        data_list[train_and_val[i]] = []
        
        which_data_path = data_path[i]
    
        for fname in sorted(os.listdir(which_data_path)):
            
            subject_data_path = os.path.join(which_data_path, fname) # fetch one h5 file from the path
            if not os.path.isfile(subject_data_path): continue 
            
            with h5py.File(subject_data_path, 'r') as data:
                if 'kspace' in data:
                    num_slice = data['kspace'].shape[0]
                else:
                    num_slice = data['kspace_4af'].shape[0]  if current_mask == 4 else data['kspace_8af'].shape[0]
                
            # the first 5 slices are mostly noise so it is better to exlude them
            data_list[train_and_val[i]] += [(fname, subject_data_path, slice) for slice in range(5, num_slice)]
    
    return data_list


In [3]:

data_path_train = './train'
data_path_val = './train'
data_list = load_data_path(data_path_train, data_path_val)

mask4 = { 'acc': 4, 'cen_fract': 0.08 }
mask8 = { 'acc': 8, 'cen_fract': 0.04 }

mask = mask4 if current_mask == 4 else mask8
acc = mask['acc']
cen_fract = mask['cen_fract']
seed = False # random masks for each slice 
num_workers = 0 # data loading is faster using a bigger number for num_workers. 0 means using one cpu to load data

# create data loader for training set. 
# It applies same to validation set as well
dataset = MRIDataset(data_list['train'], acceleration=acc, center_fraction=cen_fract, use_seed=seed)
len_dataset = len(dataset)
indx = np.arange(len_dataset)

train_indx = indx[:int(len_dataset*0.8)]
val_indx = indx[-(int(len_dataset*0.2)):]

# we divide our train dataset into 80/20 split (not randomly)
# in order to have validation dataset seperately.
train_dataset = Subset(dataset, train_indx)
val_dataset = Subset(dataset, val_indx)

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size, num_workers=num_workers) 
val_loader = DataLoader(val_dataset, shuffle=True, batch_size=batch_size, num_workers=num_workers)

dataset-loader: opening ...  ./train
dataset-loader: opening ...  ./train


In [4]:
## HELPER nested model
class ConvBlock(nn.Module):
    """
    A Convolutional Block that consists of two convolution layers each followed by
    instance normalization, relu activation and dropout.
    """

    def __init__(self, in_chans, out_chans, stride=1):
        """
        Args:
            in_chans (int): Number of channels in the input
            out_chans (int): Number of channels in the output 
        """
        super().__init__()

        self.in_chans = in_chans
        self.out_chans = out_chans
        self.stride = stride

        self.layers = nn.Sequential(
            nn.Conv2d(in_chans, out_chans, kernel_size=5, padding=2, stride=stride, bias=True),
            nn.InstanceNorm2d(out_chans),
            nn.LeakyReLU(),

            nn.Conv2d(out_chans, out_chans, kernel_size=3, padding=1, stride=1, bias=True),
            nn.InstanceNorm2d(out_chans),
            nn.LeakyReLU()
        )

    def forward(self, input):
        return self.layers(input)

class MRIModel(nn.Module):
    """
    PyTorch implementation of a U-Net mode with dense deep middle layer
    """
    def __init__(self, in_chans, out_chans, chans, num_pool_layers=4, num_depth_blocks=3):
        super().__init__()
        # test up sampling after down sampling
        self.chans = chans
        self.in_chans = in_chans
        self.out_chans = out_chans
        self.num_pool_layers = num_pool_layers
        self.num_depth_blocks = num_depth_blocks


        # First block should have no reduction in feature map size.
        # turns the inputs (2 since complex) to 32
        self.phase_head = ConvBlock(in_chans=in_chans, out_chans=chans, stride=1)
        self.down_sample_layers = nn.ModuleList([self.phase_head])

        ch = chans
        """
        First we're down sample the image while increasing the number of channels.
        Meaning smaller parts of the image across more neurons.
        Thus, extracting the important features of the image
        """
        for _ in range(num_pool_layers - 1):
            conv = ConvBlock(in_chans=ch, out_chans=ch * 2, stride=2)
            self.down_sample_layers.append(conv)
            ch *= 2

        # Size reduction happens at the beginning of a block, hence the need for stride here.
        self.mid_conv = ConvBlock(in_chans=ch, out_chans=ch, stride=2)
        self.middle_layers = nn.ModuleList()
        """
        Then we're passing the data through deep middle layers of convolutional2D.
        Adding more paramters to the network
        """
        for _ in range(num_depth_blocks - 1):
            self.middle_layers.append(ConvBlock(in_chans=ch, out_chans=ch, stride=1))

        """
        Lastly we're upsampled the image while concatinating it with previously features extracted
        by the down sampler. then passing each through layers of convolutional scan.
        Essentially emphasizing the features picked up by the down sampled version of the image.
        """
        self.up_sample_layers = nn.ModuleList()
        for _ in range(num_pool_layers - 1):
            conv = ConvBlock(in_chans=ch * 2, out_chans=ch // 2, stride=1)
            self.up_sample_layers.append(conv)
            ch //= 2
        else:  # Last block of up-sampling.
            conv = ConvBlock(in_chans=ch * 2, out_chans=ch, stride=1)
            self.up_sample_layers.append(conv)
            assert chans == ch, 'Channel indexing error!'


        # passing the resulted image through finalization process with 3 convolutional layers
        # This is to try smooth the image a bit.
        self.final_layers = nn.Sequential(
            nn.Conv2d(in_channels=ch, out_channels=ch, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=ch, out_channels=out_chans, kernel_size=1)
        )

    def forward(self, tensor):
        """
        Args:
            input (torch.Tensor): Input tensor of shape [batch_size, self.in_chans, height, width]
        Returns:
            (torch.Tensor): Output tensor of shape [batch_size, self.out_chans, height, width]
        """
        stack = list()
        output = tensor

        # Down-Sampling
        for layer in self.down_sample_layers:
            output = layer(output)
            stack.append(output)

        # Middle blocks
        output = self.mid_conv(output)
        for layer in self.middle_layers:
            output = output + layer(output)  # Residual layers in the middle.
        # Up-Sampling.
        for layer in self.up_sample_layers:
            output = F.interpolate(output, scale_factor=2, mode='bilinear', align_corners=False)
            ds_output = stack.pop()
            # concatinating with the down sample input of the same size.
            output = torch.cat([output, ds_output], dim=1)
            output = layer(output)

        final_output = self.final_layers(output)
        return final_output


#### We used a train_step generator function preseted here:

In [5]:
def generate_train_step_call(model, loss_fn, optimiser):

    # define a function inside another function
    """
        img_inputs = complex valued undersampled image
        img_target = complex valued ground truth image
    """
    def train_step(img_inputs, img_target): 
        # img_target = T.complex_abs(img_target)
        """
         permutate the image (1, w, h, 2) -> (1, 2, w, h)
         considering the complex numbers as two channel input
        """
        inputs_perm = img_und.permute(0, 3, 1, 2)

        ### foreword ###
        output_raw_pred = model(inputs_perm) # feed forward the inputs (complex image)

        # permutate the image back to its origianl shape
        img_pred_complex = output_raw_pred.permute(0, 2, 3, 1) 
        
        ### backward ###
        optimiser.zero_grad()
        # compute the loss using SSIM score between prediction and ground truth
        
        loss = loss_fn(img_target, img_pred_complex) 
        loss.backward()          # autograd = provide gradient to update the params
        optimiser.step()         # update parameters
        return loss.item()       # return the loss

    # return the newly defined function
    return train_step


#### Optimizer

The detailed optimizer code is written below.


In [6]:
epoches = 100
lr = 1e-3 # learning rate
weight_decay = 0

# create the main components to generate a train step
model = MRIModel(
    in_chans=2,
    out_chans=2,
    chans=32,
    num_pool_layers=4
).to(device)
criterion = nn.MSELoss(reduction='mean')
optimiser = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

train_step = generate_train_step_call(model, criterion, optimiser)

The optimiser we decided to use is ADAM, which is the fastest one of the bunch.
Also has better efficiency in terms memory and computional complexity. And its appropriate for problems with very noisy or sparse gradients.

We've decided to feed the network the complex numbers to have 2 channels in and out and using 4 layers for the autoencoder down and up sampling parts.


The loss function we've used is MSE. MSE is the most popular one and perform very good in most scenarios. We tried to use different loss functions such as L1, SSIM and MSE.

SSIM was a good loss funciton but wasn't very computionally efficent so we used MSE as the final choice.

### Train loop

Train loop includes an evaluation block.
We've also implemented checkpoints on each epoch iteration saving only improved models.


In [7]:
print("Training started")
losses = list()
val_losses = list()
running_loss = 0
running_valid_loss = 0
min_valid_loss = 1000
for epoch in range(epoches):
    model.train()
    for iteration, sample in enumerate(train_loader):
        img_gt, img_und, rawdata_und, masks, norm = sample
        
        # send to GPU
        img_und = img_und.to(device)
        img_gt = img_gt.to(device)
        
        loss_value = train_step(img_und, img_gt)
        # accumelators
        running_loss += loss_value
    
    # performing evalutation every epoch
    with torch.no_grad():
        model.eval()
        for iteration, sample in enumerate(val_loader):
            img_gt, img_und, rawdata_und, masks, norm = sample

            # send to GPU
            image_und = img_und.to(device)
            img_gt = img_gt.to(device)
            
            inputs = img_und.permute(0, 3, 1, 2) # passing the same way like in training

            # feed forward the inputs
            output_raw_pred = model(inputs.to(device))
            output_raw_pred = output_raw_pred.permute(0, 2, 3, 1)
            val_loss_value = criterion(img_gt, output_raw_pred)

            running_valid_loss += val_loss_value
            
    # Log the epoch loss value
    avg_loss = running_loss/len(train_loader)
    losses.append(avg_loss)
    avg_valid_loss = running_valid_loss/len(val_loader)
    
    val_losses.append(avg_valid_loss)
    print('epoch [{}/{}], loss:{:.4f}, val_loss:{:.4f}'.format(epoch+1, epoches, avg_loss, avg_valid_loss))
    running_loss = 0
    running_valid_loss = 0
    if avg_valid_loss < min_valid_loss:
        min_valid_loss = avg_valid_loss
        torch.save(model.state_dict(), PATH + ".pth")
        print("Best valid loss")
        print("------------")

Training started


KeyboardInterrupt: 



# Experiments

#### Training set, validation set and test set
We have splitted the training set in two parts: a training set for training the model and a validation set for validating the model. We validate a model focusing on the loss function, in the training process we reduce the loss function at each epoch. In the meanwhile, we need to know when to stop to avoid overfitting. To solve this problem, we  stop the training when results of the model in the validation set don't improve anymore or conversely they start to deteriorate. We will only use the test set to measure the results because the test set needs to be totally isolated of the model training and evaluation.
#### Batch size
To reduce overfitting, we train the model per batchs. A batch is set of samples. A batch size which is greater than 1 could allow us to update the gradient with the average results for each batch and not individually for each sample. By this method, we can avoid the model to memorise individual samples.
#### Dense 4-fold
In order to improve the results, we have used a variant of the u-net neural network: the dense u-net approach. In a dense u-net, we can add more layers inside the u-net process in which we apply convolutions which keep the size of the views and the number of channels. We can see an example of a dense u-net model in the figure below.

Therefore, in our model, we have added the 3 convolution layers keeping the size and number of channels. We have added these extra layers in the middle of the process of the previous model. The code of our dense u-net model is shown below.

```python
## HELPER nested model
class ConvBlock(nn.Module):
    """
    A Convolutional Block that consists of two convolution layers each followed by
    instance normalization, relu activation and dropout.
    """

    def __init__(self, in_chans, out_chans, stride=1):
        """
        Args:
            in_chans (int): Number of channels in the input
            out_chans (int): Number of channels in the output 
        """
        super().__init__()

        self.in_chans = in_chans
        self.out_chans = out_chans
        self.stride = stride

        self.layers = nn.Sequential(
            nn.Conv2d(in_chans, out_chans, kernel_size=5, padding=2, stride=stride, bias=True),
            nn.InstanceNorm2d(out_chans),
            nn.LeakyReLU(),

            nn.Conv2d(out_chans, out_chans, kernel_size=3, padding=1, stride=1, bias=True),
            nn.InstanceNorm2d(out_chans),
            nn.LeakyReLU()
        )

    def forward(self, input):
        return self.layers(input)



class MRIModel(nn.Module):
    """
    PyTorch implementation of a U-Net mode
    This is based on:
      
    """
    def __init__(self, in_chans, out_chans, chans, num_pool_layers=4, num_depth_blocks=4):
        super().__init__()
        # test up sampling after down sampling
        self.chans = chans
        self.in_chans = in_chans
        self.out_chans = out_chans
        self.drop_prob = 0.0
        self.num_pool_layers = num_pool_layers
        self.num_depth_blocks = num_depth_blocks


        # First block should have no reduction in feature map size.
        self.phase_head = ConvBlock(in_chans=in_chans, out_chans=chans, stride=1)
        self.down_sample_layers = nn.ModuleList([self.phase_head])

        ch = chans
        for _ in range(num_pool_layers - 1):
            conv = ConvBlock(in_chans=ch, out_chans=ch * 2, stride=2)
            self.down_sample_layers.append(conv)
            ch *= 2

        # Size reduction happens at the beginning of a block, hence the need for stride here.
        self.mid_conv = ConvBlock(in_chans=ch, out_chans=ch, stride=2)
        self.middle_layers = nn.ModuleList()
        for _ in range(num_depth_blocks - 1):
            self.middle_layers.append(ConvBlock(in_chans=ch, out_chans=ch, stride=1))

        self.up_sample_layers = nn.ModuleList()
        for _ in range(num_pool_layers - 1):
            conv = ConvBlock(in_chans=ch * 2, out_chans=ch // 2, stride=1)
            self.up_sample_layers.append(conv)
            ch //= 2
        else:  # Last block of up-sampling.
            conv = ConvBlock(in_chans=ch * 2, out_chans=ch, stride=1)
            self.up_sample_layers.append(conv)
            assert chans == ch, 'Channel indexing error!'

        self.final_layers = nn.Sequential(
            nn.Conv2d(in_channels=ch, out_channels=ch, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=ch, out_channels=out_chans, kernel_size=1)
        )

    def forward(self, tensor, masks):
        """
        Args:
            input (torch.Tensor): Input tensor of shape [batch_size, self.in_chans, height, width]
        Returns:
            (torch.Tensor): Output tensor of shape [batch_size, self.out_chans, height, width]
        """
        stack = list()
        output = tensor.unsqueeze(1)

        # Down-Sampling
        for layer in self.down_sample_layers:
            output = layer(output)
            stack.append(output)

        # Middle blocks
        output = self.mid_conv(output)
        for layer in self.middle_layers:
            output = output + layer(output)

        # Up-Sampling.
        for layer in self.up_sample_layers:
            output = F.interpolate(output, scale_factor=2, mode='bilinear', align_corners=False)
            ds_output = stack.pop()
            output = torch.cat([output, ds_output], dim=1) # skip connection - u-net
            output = layer(output)

        return self.final_layers(output).squeeze(0)
```

*(SSIM scores are calculated using the validation part of the dataset)*

#### Denser 4-fold

<table>
       <tr>
        <td> epoches | </td>
        <td> 3 </td>
       </tr>
    <tr>
        <td> learning rate |</td>
        <td>  1e-3 </td>
       </tr>
    <tr>
        <td> weight_decay |</td>
        <td> 0 </td>
       </tr>
    <tr>
        <td> Average SSIM score |</td>
        <td> 0.6413</td>
       </tr>
</table>



![title](https://am3pap001files.storage.live.com/y4m-bT2r4tQA6GTXJlfgLBgFG95Duh1dip7zGUdN7BsRNuX28AWZuEu9glaFiIIWiEiYFZX7gsxf6Xk1UDHjk-gojs7sK9XS33SSRGaczun_wnULHqJpGKQgPWVuNWTueRQAK9UDemIvBYCtyRgzWcM4SQgiNQBo_IKI5dFp_OzfIfYaApkkJ9hb98bOC9GGwDBEJGOPjbEeSiAMV4xGxK_hg/4af-pred-gt.png?psid=1&width=851&height=260)

The results could be percived in the figure. On the left, we have the low resolution MRI image, in the middle the prediction of our model and on the right we have the high resolution image taken as reference.

We can see also how the loss is decresing in the training.



![title](https://am3pap001files.storage.live.com/y4mMeDfCGvjt35lnQq3o9pZrXTYk0lXKazFu4bR_k1qpYe2aFPxJ3sjBz4sabaHxK5l4FLuYceMSLNnnZHkYsdKDQOtOvF-A-_ezEhdHTv_Rx7VdoNPPRWgSK3Tz5W8jg_1Dc0Fvbj7b4RzepS-ftHejkrIcw-TxtNn-meCp5B7a1kKLEmOMpEp9kd2JmjJhV5WeR3KbthJrUCr9NxKBSTJwA/loss.png?psid=1&width=380&height=248)
#### Denser early stop 4-fold

<table>
       <tr>
        <td> epoches | </td>
        <td> 3 </td>
       </tr>
    <tr>
        <td> learning rate |</td>
        <td>  1e-4</td>
       </tr>
    <tr>
        <td> weight_decay |</td>
        <td>  1e-2</td>
       </tr>
    <tr>
        <td> Average SSIM score |</td>
        <td>  0.6319</td>
       </tr>
</table>


![title](https://am3pap001files.storage.live.com/y4mZT5_xbXgPRv2LYkGAj-uXgOQQheyt_C-JYRpKV-rOAcDo1ZHjphDSwPyR2gwFmcLM2bh9jslkbeJcTBdQZ0sGATTj9b2A2OzCWgNgc2EUkZZpq4h7x5Hxu-Kb7XdqkbWMVqwUAMOJolljbNiw6XNZpXcxfQ1z2Ti0bqzy8rzzRdNW1PThD3ytTfbl5BybPIsP5QcMeQMlq8isvB4YF1pzw/4af-pred-gt.png?psid=1&width=632&height=194)

The results could be percived in the figure. On the left, we have the low resolution MRI image, in the middle the prediction of our model and on the right we have the high resolution image taken as reference.

We can see also how the loss is changing in the training.

![title](https://am3pap001files.storage.live.com/y4miBfx6nCx3tU88s71P-56cNghQ7bcAjogusVagC-5M-OZzbVT7pbTWzIhJpA9TtK01Y_9_MX60cH1Y9713fgoAf0HJjVOkHUS_cGFcV2Wa3PBWeNM-86YpkrCc-WeXERcug9hrynRBjQU9rs8JACsA7iY9CTIhhxRnJehgTWC5YrJmm6bdRDDh9R_x6bqzvfp77L-wcXYTTYpLbJprt7xKQ/loss.png?psid=1&width=380&height=248)
#### Denser early stop 8-fold

<table>
       <tr>
        <td> epoches | </td>
        <td> 3 </td>
       </tr>
    <tr>
        <td> learning rate |</td>
        <td>  1e-3 </td>
       </tr>
    <tr>
        <td> weight_decay |</td>
        <td> 0 </td>
       </tr>
    <tr>
        <td> Average SSIM score |</td>
        <td> 0.5185</td>
       </tr>
</table>


![title](https://am3pap001files.storage.live.com/y4mMpZ2z22GXnSyLTz7o8YqjXqstZpiZwR7WWwR79EyhuLDRIcPZF6idqdjOrgH_P5U29-HU6N5Ns192LBLc0QMCKdM3o9qm1cUywSbDeGCIaepPMGWuhsQ7FBYftKZpIUWZlJSebU3X7_Swu5qUov7knVPpt6teqYhN1NJ4a9bA1o4kj5_4k-4ZmKk-H0-UUM4CSdPbag2bYxJG7ubM8K-Uw/8af-pred-gt.png?psid=1&width=851&height=260)

The results could be percived in the figure. On the left, we have the low resolution MRI image, in the middle the prediction of our model and on the right we have the high resolution image taken as reference.

We can see also how the loss is decresing gerenally during the training. 

![title](https://am3pap001files.storage.live.com/y4mR-H7YXVqGigAEA8LhnTC-DEC2DrCr-9Aw0eQ7OJcWvQRWmZHPupDHUEVxWpHUk6r3L4Qn4DiH7_-WELBWhjSTf8nFYC6eeYWdVrGevbExAfqg2GRVAd9MJBOwIuRiiAH8UM3WpLTpZjZiz-e3xU9OJhnW3CZwk6y5l3aQALcmFZGSJ_E48t7cbICd2QnZlD60V8o0KCNmhLFxROweUY79w/loss.png?psid=1&width=380&height=248)
#### Down-Sampling 4-fold
Using basic autoencoder model feeding cropped (320x320) absolute valued image through the network.

<table>
       <tr>
        <td> epoches | </td>
        <td> 3 </td>
       </tr>
    <tr>
        <td> learning rate |</td>
        <td>  1e-3 </td>
       </tr>
    <tr>
        <td> weight_decay |</td>
        <td> 0 </td>
       </tr>
    <tr>
        <td> Average SSIM score |</td>
        <td> 0.6420</td>
       </tr>
</table>


![title](https://am3pap001files.storage.live.com/y4mux54aemO21NTWcmlru_5646mkyIRQq3QUxCQwqmjCUO9Ay05z4YDz0Kxy9FFfQppp21cb1xtNwS9aOgVd4Z34tFQLVYdBIBlvT3pLxHRQNnKkQCjUMX3Uuc27umpr9gnihUkRk0m8lhY1EH3UiXjejqIEbXJ2rSwG9HqSTbo0zb8-DesVnHwRRGBCJDUT-9UZAh5zop1kCkFEBs7FLKKug/loss.png?psid=1&width=380&height=248)

The results could be percived in the figure. On the left, we have the low resolution MRI image, in the middle the prediction of our model and on the right we have the high resolution image taken as reference.

![title](https://am3pap001files.storage.live.com/y4mh8UgkTSNJMRlp0UCYufJaIa3EWNShFkKHzpZbm682z2iBXajMyUmKb3Cn6Ps6hSWxo-2IUhQo354NfDCNVxgIGrfcmpkbtScdCpEsdZBeIT23gCCiBT9fkgWUZ4of7tQl6XNN57U2m55FDA7iHyoIEuNbUfF6IlrdzVhWihiUK7G86BnxSSOeP9CCTuUEqHxsFzy3ttadFtolGc89w0vHQ/und4af-pred-gt.png?psid=1&width=851&height=260)
#### U-net 8-af batch k-space correction
We wanted also to apply the k-space correction described in the design part to see if this will improve our results. We implemented it, we display some of the results.

![title](https://am3pap001files.storage.live.com/y4mIGbPG3y0SOQ2yIsQ7lM5RWWFORITbwbsNSAVqCIEkLbxzDBEZO569pwWizXsZGqd4McLQRqj79FNfbPPvf_LKuVpKTiYwdqzW1X0ewuiHWJnlHcGZtWTzHoQfs1I96FTVlOZZ3F458Btnq2DT3Vxpv6U9dLqooF7PuoMEueFaLkuRd_GyTMBBo6TSs9ZWKbEy84tGVXXyDBclZQ45rzXww/newresult.png?psid=1&width=859&height=197)

On the left we have the undersampled MRI image, the second one is the prediction of the model, the third one is the result after applying k-space correction and the final one on the right the fully sampled MRI image as reference.

*(k-space correction wasn't yielding good results because we're working on cropped image which has less information of the fully sampled k-space)*


For this U-net 8-af model, it can be clearly presented that the loss of this training tends to zero which has the best performance almost these models tested before.
![title](https://am3pap001files.storage.live.com/y4mDodF177XujY1QPe3k_kXKkcn0C4zhpL22FxzBfAVJOSK3rdrEJhjsFHRSEQ_K3Ng33jPzfiSgk1Klydzz_gMTWxqM-HxneEkCLqazegdmfnNh_z_8wZQALGZ9UYvgxB5KMSRK8yndsYWWlkmsLQF3dGy5CD_yWrhyjkKhDjqTKDX5Hpd3KP0Opq3yYlQVvpwKF4yqhBfAPyQ-UyvHvHVBw/loss.png?psid=1&width=378&height=249)

### Comparation of these different models


|test|name|epoches|learning rate|weight_decay|Average SSIM score|
|--|---|--|---|--|---|
|1|denser-4af|3|1e-3|0|0.6413|
|2|denser-4af|3|1e-4|1e-2|0.6319|
|3|denser-8af|3|1e-3|0|0.5185|
|4|downsampling-4af|3|1e-3|0|0.6420|



